# 네이버 영화 리뷰 데이터 (nsmc) 감성 분석 - LSTM,RNN,GRU

* Dataset   
    *   네이버 영화 리뷰 데이터 (naver sentiment movie corpus)
<table align="left">
  <td>
    <a target="_blank" href="https://github.com/e9t/nsmc">네이버 영화 리뷰 데이터</a>
  </td>
</table>
    



## 라이브러리 불러오기



*   모델 적용할 경우 필요한 라이브러리를 불러옵니다

    *   시각화: matplotlib,seaborn
    *   모델링: pandas,numpy,tensorflow

* 한국어 형태소 분리를 위해 Okt 적용하였습니다.
* 단어 사전을 만들기 위해서 파이썬 Counter 라이브러리 사용하였습니다.
* 분석을 위해 sklearn 라이브러리 사용하였습니다.



In [4]:
!pip install tensorflow==2.16.1

In [ ]:
!pip install konlpy

In [ ]:
# 모델링
import pandas as pd
import numpy as np
import tensorflow as tf

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn과 Counter 라이브러리
from collections import Counter
from sklearn.model_selection import train_test_split


# 한국어 형태소 Okt
from konlpy.tag import Okt

## 데이터 불러오기

* train과 test 데이터셋



In [ ]:
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")


In [ ]:
train.head()
test.head()

In [ ]:
print(f"train shape => {train.shape} \ntest shape => {test.shape}")
train.columns

## 시각화

* Seaborn의 countplot과 matplotlib의 pie chart를 적용하여 train과 test 개수를 확인 가능합니다

In [ ]:
sns.set_theme(style = "darkgrid")
ax = sns.countplot(x = 'label',data = train)

In [ ]:
sns.set_theme(style = "darkgrid")
ax = sns.countplot(x = 'label',data = test)

In [ ]:
labels,frequencies = np.unique(train.label.values, return_counts = True)
plt.figure(figsize=(5,5))
plt.pie(frequencies,labels= labels,autopct ='%1.1f%%')
plt.show()

In [ ]:
labels,frequencies = np.unique(test.label.values, return_counts = True)
plt.figure(figsize=(5,5))
plt.pie(frequencies,labels= labels,autopct ='%1.1f%%')
plt.show()

- 라벨 값이 균등하게 train과 test에서 확인할 수 있습니다.

## 결과 값 확인



In [ ]:
#학습 데이터의 결측 값
train.isnull().sum()


In [ ]:
# test 데이터의 결측 값
test.isnull().sum()

## 데이터 전처리



*   데이터 중복 제거 (drop_duplicate)
*   결측 값 제거 (dropna)
*   한국어만 추출 (정규표현식 사용)
*   한국어 토큰화 (Okt)
*   불용어 제거 (조사,구두점,접미사 종류)
*   데이터에 정수 인코딩 (text_to_sequence)
*   문장 길이 분포와 적절한 최대 문장 길이 지정
*   최대 문자 길이에 따른 패딩 추가 (pad_sequences)




In [ ]:
# 중복값 제거
train.drop_duplicates(subset =['document'],inplace=True)
test.drop_duplicates(subset = ['document'],inplace=True)
print(train)
print(test)

In [ ]:
    #결측값 제거
    train = train.dropna()
    test = test.dropna()
    print(train)
    print(test)

In [ ]:
# 한국어만 추출
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train['document'].replace('', np.nan, inplace=True)

test['document'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['document'].replace('', np.nan, inplace=True)


In [ ]:
# 한국어 토큰화
okt = Okt()

# 불용어 제거
stopwords = ['요', '도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

# 토큰화 적용
train['tokenized'] = train['document'].apply(okt.morphs)
train['tokenized'] = train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


In [ ]:
test['tokenized'] = test['document'].apply(okt.morphs)
test['tokenized'] = test['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


In [ ]:
X_train = train['tokenized']
y_train = train['label']
X_test= test['tokenized']
y_test = test['label']

In [ ]:
# print(X_train)
# print(y_train)
# print(X_test)
# print(y_test)


# X_train = pd.DataFrame(X_train)
# X_train.to_csv('train_tokenized.csv')
# y_train.to_csv('train_label.csv')
# X_test.to_csv('test_tokenized.csv',encoding='utf-8-sig')
# y_test.to_csv('test_label.csv')




In [ ]:

file_path = 'train_tokenized.csv'
data = pd.read_csv(file_path)
data

In [ ]:
# 문자열 데이터를 빈도수 기준으로 단어 집합을 생성
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
# 맵핑하는 함수 text_to_sequences 적용
X_train_sequence= tokenizer.texts_to_sequences(X_train)

In [ ]:
threshold = 2
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)



In [ ]:
# 일반적으로 0번 토큰(padding)과 1번 토큰 (OOV)를 피하기 위해 2부터 시작
vocab_size = total_cnt- rare_cnt +2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
X_val= tokenizer.texts_to_sequences(X_test)

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 80 #Padding 80으로
X_train = pad_sequences(X_train_sequence, maxlen = max_len)
X_val = pad_sequences(X_val, maxlen = max_len)

sum =0
for i in X_train:
    sum+=1
print(sum)

In [ ]:
y_train

## GRU 모델 이용한 감성 분석



In [ ]:
#필용한 라이브러리 불러옵니다
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

In [ ]:
# GRU 모델을 적용
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history_GRU = model.fit(X_train, y_train, epochs=15, batch_size=60, validation_split=0.2)

## LSTM 모델을 아용한 감성 분석

In [ ]:
# LSTM 모델을 적용
from tensorflow.keras.layers import LSTM, Bidirectional

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, 100))
model_lstm.add(LSTM(100)) #Bidirectional drop
model_lstm.add(Dense(1, activation='relu')) #activation = sigmoid -> relu

In [ ]:
model_lstm.summary()

In [ ]:
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_LSTM = model_lstm.fit(X_train, y_train, epochs=15, batch_size=60, validation_split=0.2)

## Test 데이터셋에 모델 평가



*   loss 값과 accuracy값이 출력



In [ ]:
#LSTM로 학습된 모델로 평가
predict_LSTM = model_lstm.evaluate(X_test,y_test,verbose=1)
print(predict_LSTM)

In [ ]:
history_dict_LSTM = history_LSTM.history
print(history_dict_LSTM.keys())

In [ ]:
acc = history_dict_LSTM['accuracy']
val_acc = history_dict_LSTM['val_accuracy']
loss = history_dict_LSTM['loss']
val_loss =history_dict_LSTM['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs,loss, 'r', label='Training loss')
plt.plot(epochs, val_loss,'b',label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.lengend()

plt.show()

In [ ]:
#GRU로 학습된 모델로 평가
predict_GRU = model.evaluate(X_test,y_test,verbose=1)
print(predict_GRU)

In [ ]:
history_dict_GRU = history_GRU.history
print(history_dict_GRU.keys())

In [ ]:
acc = history_dict_GRU['accuracy']
val_acc = history_dict_GRU['val_accuracy']
loss = history_dict_GRU['loss']
val_loss =history_dict_GRU['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs,loss, 'r', label='Training loss')
plt.plot(epochs, val_loss,'b',label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.lengend()

plt.show()